In [1]:
import pandas as pd

In [9]:
# note data is xlsx excel format not csv

# i am going to do this in "unclean" way
sheets_dict = pd.read_excel('PD - NBA Results.xlsx', sheet_name=None)
full_table = pd.DataFrame()
for name, sheet in sheets_dict.items():
    if name != "Team List":
        sheet['sheet'] = name
        sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
        full_table = full_table.append(sheet)
    else:
        team_list = sheet

full_table.reset_index(inplace=True, drop=True)

In [10]:
full_table.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes,sheet
0,Tue Oct 22 2019,8:00p,New Orleans Pelicans,122.0,Toronto Raptors,130.0,Box Score,OT,20787.0,NaN,October Results
1,Tue Oct 22 2019,10:30p,Los Angeles Lakers,102.0,Los Angeles Clippers,112.0,Box Score,NaN,19068.0,NaN,October Results
2,Wed Oct 23 2019,7:00p,Chicago Bulls,125.0,Charlotte Hornets,126.0,Box Score,NaN,15424.0,NaN,October Results
3,Wed Oct 23 2019,7:00p,Detroit Pistons,119.0,Indiana Pacers,110.0,Box Score,NaN,17923.0,NaN,October Results
4,Wed Oct 23 2019,7:00p,Cleveland Cavaliers,85.0,Orlando Magic,94.0,Box Score,NaN,18846.0,NaN,October Results


In [28]:
full_table.shape

(726, 11)

In [11]:
team_list.head()

,Team,Divison,Conference
0,Atlanta Hawks,Southeast,Eastern
1,Boston Celtics,Atlantic,Eastern
2,Brooklyn Nets,Atlantic,Eastern
3,Charlotte Hornets,Southeast,Eastern
4,Chicago Bulls,Central,Eastern


In [42]:
team_stats = {i:{"W":0, "L":0, "Conf":[0,0], "Home":[0,0], "Away":[0,0], "streak":[], "conference":j} for i,j in zip(team_list["Team"].values, team_list["Conference"].values)}

In [43]:
# i will not do this in pandas dataframe level, but in iterative approach
for index, row in full_table.iterrows():
    visit, v_s, home, h_s = row["Visitor/Neutral"], row["PTS"], row["Home/Neutral"], row["PTS.1"]
    #print(visit, v_s, home, h_s)
    if v_s > h_s: # visitor won
        team_stats[visit]["W"] += 1
        team_stats[visit]["Away"][0] += 1
        team_stats[visit]["streak"] += ["W"]
        team_stats[home]["L"] += 1
        team_stats[home]["Home"][1] += 1
        team_stats[home]["streak"] += ["L"]
        if team_stats[visit]["conference"] == team_stats[home]["conference"]:
            team_stats[visit]["Conf"][0] += 1
            team_stats[home]["Conf"][1] += 1
    elif v_s < h_s: # home won
        team_stats[visit]["L"] += 1
        team_stats[visit]["Away"][1] += 1
        team_stats[visit]["streak"] += ["L"]
        team_stats[home]["W"] += 1
        team_stats[home]["Home"][0] += 1
        team_stats[home]["streak"] += ["W"]
        if team_stats[visit]["conference"] == team_stats[home]["conference"]:
            team_stats[visit]["Conf"][1] += 1
            team_stats[home]["Conf"][0] += 1

In [44]:
from collections import Counter
western = []
for team in team_stats.keys():
    if team_stats[team]["conference"] == "Western":
        last10 = team_stats[team]["streak"][-10:]
        c = Counter(last10)
        streak, count = last10[-1], 1
        for i in range(9):
            if last10[-(i+2)] == streak:
                count += 1
            else:
                break
        western += [[team, team_stats[team]["W"], team_stats[team]["L"], 
                    round(team_stats[team]["W"]/(team_stats[team]["W"] + team_stats[team]["L"]), 3),
                    "-".join(str(x) for x in team_stats[team]["Conf"]),
                    "-".join(str(x) for x in team_stats[team]["Home"]),
                    "-".join(str(x) for x in team_stats[team]["Away"]),
                    "-".join(str(x) for x in [c["W"], c["L"]]),
                    streak+str(count)
                   ]]

In [45]:
for i in western:
    print(i)

['Dallas Mavericks', 22, 13, 0.629, '13-6', '10-8', '12-5', '5-5', 'L1']
['Denver Nuggets', 24, 11, 0.686, '14-6', '15-4', '9-7', '7-3', 'L1']
['Golden State Warriors', 9, 28, 0.243, '7-20', '6-12', '3-16', '4-6', 'L4']
['Houston Rockets', 24, 11, 0.686, '14-7', '13-4', '11-7', '7-3', 'W2']
['Los Angeles Clippers', 25, 12, 0.676, '16-9', '15-4', '10-8', '5-5', 'L1']
['Los Angeles Lakers', 28, 7, 0.8, '20-4', '13-4', '15-3', '6-4', 'W4']
['Memphis Grizzlies', 14, 22, 0.389, '9-14', '7-12', '7-10', '5-5', 'W1']
['Minnesota Timberwolves', 13, 21, 0.382, '6-16', '5-11', '8-10', '3-7', 'W1']
['New Orleans Pelicans', 12, 24, 0.333, '10-15', '6-11', '6-13', '6-4', 'W1']
['Oklahoma City Thunder', 20, 15, 0.571, '14-11', '12-6', '8-9', '9-1', 'W5']
['Phoenix Suns', 14, 21, 0.4, '9-17', '7-12', '7-9', '3-7', 'W1']
['Portland Trail Blazers', 15, 21, 0.417, '9-15', '8-9', '7-12', '5-5', 'W1']
['Sacramento Kings', 13, 23, 0.361, '9-14', '7-10', '6-13', '1-9', 'L1']
['San Antonio Spurs', 14, 20, 0.4

In [46]:
# column names
cols = ["Team", "W", "L", "Pct", "Conf", "Home", "Away", "L10", "Strk"]

In [54]:
Western = pd.DataFrame(data=western, columns=cols)
Western['Rank'] = Western['Pct'].rank(method='max', ascending=False)
Western = Western.astype({"Rank": "int"})

In [55]:
Western.head(20)

,Team,W,L,Pct,Conf,Home,Away,L10,Strk,Rank
0,Dallas Mavericks,22,13,0.629,13-6,10-8,12-5,5-5,L1,6
1,Denver Nuggets,24,11,0.686,14-6,15-4,9-7,7-3,L1,3
2,Golden State Warriors,9,28,0.243,7-20,6-12,3-16,4-6,L4,15
3,Houston Rockets,24,11,0.686,14-7,13-4,11-7,7-3,W2,3
4,Los Angeles Clippers,25,12,0.676,16-9,15-4,10-8,5-5,L1,4
5,Los Angeles Lakers,28,7,0.800,20-4,13-4,15-3,6-4,W4,1
6,Memphis Grizzlies,14,22,0.389,9-14,7-12,7-10,5-5,W1,11
7,Minnesota Timberwolves,13,21,0.382,6-16,5-11,8-10,3-7,W1,12
8,New Orleans Pelicans,12,24,0.333,10-15,6-11,6-13,6-4,W1,14
9,Oklahoma City Thunder,20,15,0.571,14-11,12-6,8-9,9-1,W5,7


In [56]:
eastern = []
for team in team_stats.keys():
    if team_stats[team]["conference"] == "Eastern":
        last10 = team_stats[team]["streak"][-10:]
        c = Counter(last10)
        streak, count = last10[-1], 1
        for i in range(9):
            if last10[-(i+2)] == streak:
                count += 1
            else:
                break
        eastern += [[team, team_stats[team]["W"], team_stats[team]["L"], 
                    round(team_stats[team]["W"]/(team_stats[team]["W"] + team_stats[team]["L"]), 3),
                    "-".join(str(x) for x in team_stats[team]["Conf"]),
                    "-".join(str(x) for x in team_stats[team]["Home"]),
                    "-".join(str(x) for x in team_stats[team]["Away"]),
                    "-".join(str(x) for x in [c["W"], c["L"]]),
                    streak+str(count)
                   ]]

In [59]:
Eastern = pd.DataFrame(data=eastern, columns=cols)
Eastern['Rank'] = Eastern['Pct'].rank(method='max', ascending=False)
Eastern = Eastern.astype({"Rank": "int"})

In [60]:
Eastern.head(20)

,Team,W,L,Pct,Conf,Home,Away,L10,Strk,Rank
0,Atlanta Hawks,8,28,0.222,5-19,4-12,4-16,2-8,W1,15
1,Boston Celtics,25,8,0.758,18-5,14-2,11-6,8-2,W3,2
2,Brooklyn Nets,16,18,0.471,10-9,9-7,7-11,3-7,L5,7
3,Charlotte Hornets,15,23,0.395,10-14,7-11,8-12,3-7,W2,9
4,Chicago Bulls,13,23,0.361,9-15,7-13,6-10,4-6,L3,11
5,Cleveland Cavaliers,10,25,0.286,6-21,6-12,4-13,4-6,L3,14
6,Detroit Pistons,13,23,0.361,9-18,8-10,5-13,2-8,W1,11
7,Indiana Pacers,22,14,0.611,15-10,15-4,7-10,5-5,L2,6
8,Miami Heat,25,10,0.714,18-4,16-1,9-9,7-3,L1,3
9,Milwaukee Bucks,32,5,0.865,20-3,18-2,14-3,8-2,W5,1


In [61]:
# let's save to one excel file
writer = pd.ExcelWriter('final_output.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
Western.to_excel(writer, sheet_name='WesternConferenceStandings')
Eastern.to_excel(writer, sheet_name='EasternConferenceStandings')

writer.save()